In [1]:
import mysql.connector
import pandas as pd

In [8]:
csv_files = [(
'train.csv','train')]

In [9]:
conn = mysql.connector.connect(
    host = "127.0.0.1",
    port = "3306",
    user = "root",
    password = "Gtriyar@2712",
    database = "sales_end_project"
)

In [10]:
cursor= conn.cursor()
import os

In [11]:
folder_path = 'C:/Users/gracy/Downloads/Sales end to end'

In [12]:
def get_sql_type(dtype):
    if pd.api.types.is_integer_dtype(dtype):
        return 'INT'
    elif pd.api.types.is_float_dtype(dtype):
        return 'FLOAT'
    elif pd.api.types.is_bool_dtype(dtype):
        return 'BOOLEAN'
    elif pd.api.types.is_datetime64_any_dtype(dtype):
        return 'DATETIME'
    else:
        return 'TEXT'

for csv_file, table_name in csv_files:
    file_path = os.path.join(folder_path, csv_file)
    
    # Read the CSV file into a pandas DataFrame
    df = pd.read_csv(file_path)
    
    # Replace NaN with None to handle SQL NULL
    df = df.where(pd.notnull(df), None)
    
    # Debugging: Check for NaN values
    print(f"Processing {csv_file}")
    print(f"NaN values before replacement:\n{df.isnull().sum()}\n")

    # Clean column names
    df.columns = [col.replace(' ', '_').replace('-', '_').replace('.', '_') for col in df.columns]

    # Generate the CREATE TABLE statement with appropriate data types
    columns = ', '.join([f'`{col}` {get_sql_type(df[col].dtype)}' for col in df.columns])
    create_table_query = f'CREATE TABLE IF NOT EXISTS `{table_name}` ({columns})'
    cursor.execute(create_table_query)

    # Insert DataFrame data into the MySQL table
    for _, row in df.iterrows():
        # Convert row to tuple and handle NaN/None explicitly
        values = tuple(None if pd.isna(x) else x for x in row)
        sql = f"INSERT INTO `{table_name}` ({', '.join(['`' + col + '`' for col in df.columns])}) VALUES ({', '.join(['%s'] * len(row))})"
        cursor.execute(sql, values)

    # Commit the transaction for the current CSV file
    conn.commit()

# Close the connection
conn.close()

Processing train.csv
NaN values before replacement:
Row ID            0
Order ID          0
Order Date        0
Ship Date         0
Ship Mode         0
Customer ID       0
Customer Name     0
Segment           0
Country           0
City              0
State             0
Postal Code      11
Region            0
Product ID        0
Category          0
Sub-Category      0
Product Name      0
Sales             0
dtype: int64



In [2]:
import pandas as  pd
import numpy as np
import mysql.connector 
import matplotlib.pyplot as plt
import seaborn as sns
import warnings 
warnings.filterwarnings('ignore')

db =  mysql.connector.connect(
    host = "127.0.0.1",
    port = "3306",
    user = "root",
    password = "Gtriyar@2712",
    database = "sales_end_project"
)

cur = db.cursor()
query="SELECT * FROM sales_data;"
df=pd.read_sql(query,db)
df.head()



,Row_ID,Order_ID,Ship_Mode,Customer_ID,Customer_Name,Segment,Country,City,State,Postal_Code,Region,Product_ID,Category,Sub_Category,Product_Name,Sales,Order_Date,Ship_Date
0,1,CA-2017-152156,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420.0,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.960,2017-11-08,2017-11-11
1,2,CA-2017-152156,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420.0,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.940,2017-11-08,2017-11-11
2,3,CA-2017-138688,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,California,90036.0,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,14.620,2017-06-12,2017-06-16
3,4,US-2016-108966,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,Florida,33311.0,South,FUR-TA-10000577,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,957.578,2016-10-11,2016-10-18
4,5,US-2016-108966,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,Florida,33311.0,South,OFF-ST-10000760,Office Supplies,Storage,Eldon Fold 'N Roll Cart System,22.368,2016-10-11,2016-10-18


In [3]:
# Shape
print("Rows:", df.shape[0], " Columns:", df.shape[1])

# Missing values
print("\nMissing Values:")
print(df.isnull().sum())

# Unique counts per column
print("\nUnique Values:")
print(df.nunique())

# Descriptive stats for numeric cols
print("\nNumeric Summary:")
print(df.describe())

# Descriptive stats for categorical cols
print("\nCategorical Summary:")
print(df.describe(include='object'))

print(df.dtypes)


Rows: 9800  Columns: 18

Missing Values:
Row_ID            0
Order_ID          0
Ship_Mode         0
Customer_ID       0
Customer_Name     0
Segment           0
Country           0
City              0
State             0
Postal_Code      11
Region            0
Product_ID        0
Category          0
Sub_Category      0
Product_Name      0
Sales             0
Order_Date        0
Ship_Date         0
dtype: int64

Unique Values:
Row_ID           9800
Order_ID         4922
Ship_Mode           4
Customer_ID       793
Customer_Name     793
Segment             3
Country             1
City              529
State              49
Postal_Code       626
Region              4
Product_ID       1861
Category            3
Sub_Category       17
Product_Name     1849
Sales            5755
Order_Date       1230
Ship_Date        1326
dtype: int64

Numeric Summary:
            Row_ID   Postal_Code         Sales
count  9800.000000   9789.000000   9800.000000
mean   4900.500000  55273.322403    230.769068
st

Revenue & Sales performance

In [6]:
# what is the total revenue over diff time period(daily,monthly,yearly)?
import pandas as  pd
import numpy as np
import mysql.connector 
import matplotlib.pyplot as plt
import seaborn as sns
import warnings 
warnings.filterwarnings('ignore')

conn =  mysql.connector.connect(
    host = "127.0.0.1",
    port = "3306",
    user = "root",
    password = "Gtriyar@2712",
    database = "sales_end_project"
)

cur = conn.cursor(buffered=True)

query = """
select year(order_date) as Year,month(order_Date)as Months , round(sum(sales),2) as total_sales
from sales_data
group by year(order_Date),month(order_date)
order by year(order_date),month(order_date);
"""
cur.execute(query)
data = cur.fetchall()
df = pd.DataFrame(data, columns = ["Year","Months","Total_Sales"])
df


,Year,Months,Total_Sales
0,2015,1,14205.71
1,2015,2,4519.89
2,2015,3,55205.80
3,2015,4,27906.85
4,2015,5,23644.30
5,2015,6,34322.94
6,2015,7,33781.54
7,2015,8,27117.54
8,2015,9,81623.53
9,2015,10,31453.39


In [7]:
# How has revenue grown or decline over time?
import pandas as  pd
import numpy as np
import mysql.connector 
import matplotlib.pyplot as plt
import seaborn as sns
import warnings 
warnings.filterwarnings('ignore')

conn =  mysql.connector.connect(
    host = "127.0.0.1",
    port = "3306",
    user = "root",
    password = "Gtriyar@2712",
    database = "sales_end_project"
)

cur = conn.cursor(buffered=True)

query = """
select year , sales, prev_sales , ROUND(((sales - prev_sales) / prev_sales) * 100, 2) as percentage_change
from (
SELECT 
    YEAR(order_date) AS year,
    ROUND(SUM(sales),2) AS sales,
    LAG(ROUND(SUM(sales),2)) OVER (ORDER BY YEAR(order_date)) AS prev_sales
FROM sales_data
GROUP BY YEAR(order_date)
ORDER BY year) as sub;
"""
cur.execute(query)
data = cur.fetchall()
df = pd.DataFrame(data, columns = ["Year","Sales","Prev_Sales","%change"])
df

,Year,Sales,Prev_Sales,%change
0,2015,479856.21,NaN,NaN
1,2016,459436.01,479856.21,-4.26
2,2017,600192.55,459436.01,30.64
3,2018,722052.02,600192.55,20.30


In [8]:
# Which segments(Consumer , Corporate, Home Office) contribute most to revenue?
import pandas as  pd
import numpy as np
import mysql.connector 
import matplotlib.pyplot as plt
import seaborn as sns
import warnings 
warnings.filterwarnings('ignore')

conn =  mysql.connector.connect(
    host = "127.0.0.1",
    port = "3306",
    user = "root",
    password = "Gtriyar@2712",
    database = "sales_end_project"
)

cur = conn.cursor(buffered=True)
query = """
select segment , round(sum(sales),2) as total_Revenue
from sales_data
group by segment
order by round(sum(sales),2) desc;
"""
cur.execute(query)
data = cur.fetchall()
df = pd.DataFrame(data, columns = ["Segment","Sales"])
df

,Segment,Sales
0,Consumer,1148060.53
1,Corporate,688494.07
2,Home Office,424982.18


In [9]:

import pandas as  pd
import numpy as np
import mysql.connector 
import matplotlib.pyplot as plt
import seaborn as sns
import warnings 
warnings.filterwarnings('ignore')

conn =  mysql.connector.connect(
    host = "127.0.0.1",
    port = "3306",
    user = "root",
    password = "Gtriyar@2712",
    database = "sales_end_project"
)

cur = conn.cursor(buffered=True)
query = """
select ship_mode, round(sum(sales),2) as total_Revenue
from sales_data
group by ship_mode
order by round(sum(sales),2) desc;
"""
cur.execute(query)
data = cur.fetchall()
df = pd.DataFrame(data, columns = ["Ship_Mode","Total_Revenue"])
df

,Ship_Mode,Total_Revenue
0,Standard Class,1340831.31
1,Second Class,449914.18
2,First Class,345572.26
3,Same Day,125219.04


In [12]:
import pandas as  pd
import numpy as np
import mysql.connector 
import matplotlib.pyplot as plt
import seaborn as sns
import warnings 
warnings.filterwarnings('ignore')

conn =  mysql.connector.connect(
    host = "127.0.0.1",
    port = "3306",
    user = "root",
    password = "Gtriyar@2712",
    database = "sales_end_project"
)

cur = conn.cursor(buffered=True)
query = """
SELECT 
    month,
    ROUND(AVG(monthly_sales),2) AS avg_sales
FROM (
    SELECT 
        YEAR(order_date) AS year,
        MONTH(order_date) AS month,
        SUM(sales) AS monthly_sales
    FROM sales_data
    GROUP BY YEAR(order_date), MONTH(order_date)
) sub
GROUP BY month
ORDER BY month;
"""
cur.execute(query)
data = cur.fetchall()
df = pd.DataFrame(data, columns = ["Months","Avg_Sales"])
df

,Months,Avg_Sales
0,1,23572.91
1,2,14842.78
2,3,49393.40
3,4,34070.75
4,5,38521.68
5,6,36459.38
6,7,36383.92
7,8,39328.98
8,9,75025.85
9,10,49874.07


PRODUCT & CATEGORY ANALYSIS

In [17]:
#  Which products and categories generate the most revenue
import pandas as  pd
import numpy as np
import mysql.connector 
import matplotlib.pyplot as plt
import seaborn as sns
import warnings 
warnings.filterwarnings('ignore')

conn =  mysql.connector.connect(
    host = "127.0.0.1",
    port = "3306",
    user = "root",
    password = "Gtriyar@2712",
    database = "sales_end_project"
)

cur = conn.cursor(buffered=True)
query = """
select category , round(sum(sales),2) as total_revenue
from sales_data
group by category;
"""
cur.execute(query)
data = cur.fetchall()
df = pd.DataFrame(data, columns = ["Category","Total_Revenue"])
df

,Category,Total_Revenue
0,Furniture,728658.57
1,Office Supplies,705422.33
2,Technology,827455.87


In [19]:
import pandas as  pd
import numpy as np
import mysql.connector 
import matplotlib.pyplot as plt
import seaborn as sns
import warnings 
warnings.filterwarnings('ignore')

conn =  mysql.connector.connect(
    host = "127.0.0.1",
    port = "3306",
    user = "root",
    password = "Gtriyar@2712",
    database = "sales_end_project"
)

cur = conn.cursor(buffered=True)
query = """
select product_id,product_name, round(sum(sales),2) as Total_Sales
from sales_data
group by product_id , product_name
order by round(sum(sales),2);
"""
cur.execute(query)
data = cur.fetchall()
df = pd.DataFrame(data, columns = ["Sub_Category","Total_Revenue","Revenue_%"])
df

,Sub_Category,Total_Revenue,Revenue_%
0,OFF-AP-10002203,Eureka Disposable Bags for Sanitaire Vibra Gro...,1.62
1,OFF-LA-10003388,Avery 5,5.76
2,OFF-PA-10000048,Xerox 20,6.48
3,OFF-EN-10001535,Grip Seal Envelopes,7.07
4,OFF-SU-10003936,Acme Serrated Blade Letter Opener,7.63
...,...,...,...
1888,OFF-BI-10001359,GBC DocuBind TL300 Electric Binding System,19823.48
1889,FUR-CH-10002024,HON 5400 Series Task Chairs for Big and Tall,21870.58
1890,TEC-MA-10002412,Cisco TelePresence System EX90 Videoconferenci...,22638.48
1891,OFF-BI-10003527,Fellowes PB500 Electric Punch Plastic Comb Bin...,27453.38


Customer Analysis

In [26]:
#who are the top 10 customers by revenue
import pandas as  pd
import numpy as np
import mysql.connector 
import matplotlib.pyplot as plt
import seaborn as sns
import warnings 
warnings.filterwarnings('ignore')

conn =  mysql.connector.connect(
    host = "127.0.0.1",
    port = "3306",
    user = "root",
    password = "Gtriyar@2712",
    database = "sales_end_project"
)

cur = conn.cursor(buffered=True)
query= """
select customer_id , customer_name, round(sum(sales),2) as Revenue
from sales_data
group by customer_id , customer_name
order by round(sum(sales),2) desc
limit 10;
"""
cur.execute(query)
data = cur.fetchall()
df = pd.DataFrame(data, columns = ["Customer_id","Customer_name","Revenue"])
df

,Customer_id,Customer_name,Revenue
0,SM-20320,Sean Miller,25043.05
1,TC-20980,Tamara Chand,19052.22
2,RB-19360,Raymond Buch,15117.34
3,TA-21385,Tom Ashbrook,14595.62
4,AB-10105,Adrian Barton,14473.57
5,KL-16645,Ken Lonsdale,14175.23
6,SC-20095,Sanjit Chand,14142.33
7,HL-15040,Hunter Lopez,12873.30
8,SE-20110,Sanjit Engle,12209.44
9,CC-12370,Christopher Conant,12129.07


In [27]:
#  most loyal/ repeat customers?
import pandas as  pd
import numpy as np
import mysql.connector 
import matplotlib.pyplot as plt
import seaborn as sns
import warnings 
warnings.filterwarnings('ignore')

conn =  mysql.connector.connect(
    host = "127.0.0.1",
    port = "3306",
    user = "root",
    password = "Gtriyar@2712",
    database = "sales_end_project"
)

cur = conn.cursor(buffered=True)
query= """
SELECT 
    customer_id, customer_name, COUNT(DISTINCT order_id) as Totals
FROM
    sales_data
GROUP BY customer_id , customer_name
HAVING COUNT(DISTINCT order_id) > 1
ORDER BY COUNT(DISTINCT order_id) DESC
LIMIT 10;
"""
cur.execute(query)
data = cur.fetchall()
df = pd.DataFrame(data, columns = ["Customer_id","Customer_name","Total_order"])
df

,Customer_id,Customer_name,Total_order
0,EP-13915,Emily Phan,17
1,CK-12205,Chloris Kastensmidt,13
2,JE-15745,Joel Eaton,13
3,PG-18820,Patrick Gardner,13
4,ZC-21910,Zuschuss Carroll,13
5,AH-10690,Anna Häberlin,12
6,BD-11320,Bill Donatelli,12
7,BP-11095,Bart Pistole,12
8,CL-12565,Clay Ludtke,12
9,CS-12250,Chris Selesnick,12


In [29]:
# avg order value per customer?
import pandas as  pd
import numpy as np
import mysql.connector 
import matplotlib.pyplot as plt
import seaborn as sns
import warnings 
warnings.filterwarnings('ignore')

conn =  mysql.connector.connect(
    host = "127.0.0.1",
    port = "3306",
    user = "root",
    password = "Gtriyar@2712",
    database = "sales_end_project"
)

cur = conn.cursor(buffered=True)
query= """
SELECT 
    customer_id,
    ROUND(SUM(sales) / COUNT(DISTINCT order_id), 2) AS avg_order_value
FROM sales_data
GROUP BY customer_id
ORDER BY avg_order_value DESC;

"""
cur.execute(query)
data = cur.fetchall()
df = pd.DataFrame(data, columns = ["Customer_id","AOV"])
df

,Customer_id,AOV
0,SM-20320,5008.61
1,TC-20980,3810.44
2,TA-21385,3648.90
3,GT-14635,3117.07
4,BM-11140,2947.41
...,...,...
788,SG-20890,15.98
789,SC-20845,14.11
790,RS-19870,11.16
791,LD-16855,5.30


 Geography & Regional Insights?

In [30]:
# Region which generates the highest sale
import pandas as  pd
import numpy as np
import mysql.connector 
import matplotlib.pyplot as plt
import seaborn as sns
import warnings 
warnings.filterwarnings('ignore')

conn =  mysql.connector.connect(
    host = "127.0.0.1",
    port = "3306",
    user = "root",
    password = "Gtriyar@2712",
    database = "sales_end_project"
)

cur = conn.cursor(buffered=True)
query= """
SELECT 
    region,
    ROUND(SUM(sales), 2) AS total_sales
FROM sales_data
GROUP BY region
ORDER BY total_sales DESC;

"""
cur.execute(query)
data = cur.fetchall()
df = pd.DataFrame(data, columns = ["Region","Total_Sales"])
df

,Customer_id,AOV
0,West,710219.68
1,East,669518.72
2,Central,492646.91
3,South,389151.46


In [31]:
#  company should expand marketing or inventory efforts based on revenue patterns?
import pandas as  pd
import numpy as np
import mysql.connector 
import matplotlib.pyplot as plt
import seaborn as sns
import warnings 
warnings.filterwarnings('ignore')

conn =  mysql.connector.connect(
    host = "127.0.0.1",
    port = "3306",
    user = "root",
    password = "Gtriyar@2712",
    database = "sales_end_project"
)

cur = conn.cursor(buffered=True)
query= """
SELECT 
    region,
    ROUND(SUM(sales), 2) AS total_sales,
    COUNT(DISTINCT customer_id) AS total_customers,
    ROUND(SUM(sales) / COUNT(DISTINCT customer_id), 2) AS revenue_per_customer
FROM sales_data
GROUP BY region
ORDER BY total_sales DESC;

"""
cur.execute(query)
data = cur.fetchall()
df = pd.DataFrame(data, columns = ["Region","Total_Sales","Total Customer","Revenue per customer"])
df

,Region,Total_Sales,Total Customer,Revenue per customer
0,West,710219.68,681,1042.91
1,East,669518.72,669,1000.78
2,Central,492646.91,626,786.98
3,South,389151.46,509,764.54


Operational / Delivery Analysis

In [34]:
 # avg shipping time per ship mode?
import pandas as  pd
import numpy as np
import mysql.connector 
import matplotlib.pyplot as plt
import seaborn as sns
import warnings 
warnings.filterwarnings('ignore')

conn =  mysql.connector.connect(
    host = "127.0.0.1",
    port = "3306",
    user = "root",
    password = "Gtriyar@2712",
    database = "sales_end_project"
)

cur = conn.cursor(buffered=True)
query= """
SELECT 
    ship_mode,
    ROUND(AVG(DATEDIFF(ship_date, order_date)), 2) AS avg_shipping_time
FROM
    sales_data
GROUP BY ship_mode
ORDER BY ROUND(AVG(DATEDIFF(ship_date, order_date)), 2);

"""
cur.execute(query)
data = cur.fetchall()
df = pd.DataFrame(data, columns = ["Ship Mode","avg_ship_time"])
df

,Ship Mode,avg_ship_time
0,Same Day,0.04
1,First Class,2.18
2,Second Class,3.25
3,Standard Class,5.01


In [35]:
# Which ship modes are most cost- effective vs fast?
import pandas as  pd
import numpy as np
import mysql.connector 
import matplotlib.pyplot as plt
import seaborn as sns
import warnings 
warnings.filterwarnings('ignore')

conn =  mysql.connector.connect(
    host = "127.0.0.1",
    port = "3306",
    user = "root",
    password = "Gtriyar@2712",
    database = "sales_end_project"
)

cur = conn.cursor(buffered=True)
query= """
SELECT 
    Ship_Mode,
    ROUND(AVG(DATEDIFF(Ship_Date, Order_Date)), 2) AS avg_shipping_days,
    ROUND(SUM(Sales), 2) AS total_revenue
FROM sales_data
GROUP BY Ship_Mode
ORDER BY total_revenue DESC;
"""
cur.execute(query)
data = cur.fetchall()
df = pd.DataFrame(data, columns = ["Ship Mode","avg_ship_time","Total_Revenue"])
df

,Ship Mode,avg_ship_time,Total_Revenue
0,Standard Class,5.01,1340831.31
1,Second Class,3.25,449914.18
2,First Class,2.18,345572.26
3,Same Day,0.04,125219.04


Forecasting & Strategy 

In [37]:
# next month's or next quarter sales using trends?
import pandas as  pd
import numpy as np
import mysql.connector 
import matplotlib.pyplot as plt
import seaborn as sns
import warnings 
warnings.filterwarnings('ignore')

conn =  mysql.connector.connect(
    host = "127.0.0.1",
    port = "3306",
    user = "root",
    password = "Gtriyar@2712",
    database = "sales_end_project"
)

cur = conn.cursor(buffered=True)
query= """
SELECT
    DATE_FORMAT(order_date, '%Y-%m') AS month,
    round(SUM(sales)) AS total_sales
FROM sales_data
GROUP BY month
ORDER BY month;
"""
cur.execute(query)
data = cur.fetchall()
df = pd.DataFrame(data, columns = ["month","Total_Sales"])
df

,month,Total_Sales
0,2015-01,14206.0
1,2015-02,4520.0
2,2015-03,55206.0
3,2015-04,27907.0
4,2015-05,23644.0
5,2015-06,34323.0
6,2015-07,33782.0
7,2015-08,27118.0
8,2015-09,81624.0
9,2015-10,31453.0


In [38]:
#products or regions show growth potential?
import pandas as  pd
import numpy as np
import mysql.connector 
import matplotlib.pyplot as plt
import seaborn as sns
import warnings 
warnings.filterwarnings('ignore')

conn =  mysql.connector.connect(
    host = "127.0.0.1",
    port = "3306",
    user = "root",
    password = "Gtriyar@2712",
    database = "sales_end_project"
)

cur = conn.cursor(buffered=True)
query= """
SELECT category, 
       round(SUM(sales),2)
       AS total_sales
FROM sales_data
GROUP BY category
ORDER BY total_sales DESC;
SELECT region, 
       round(SUM(sales),2) AS total_sales
FROM sales_data
GROUP BY region
ORDER BY total_sales DESC;
"""
cur.execute(query)
data = cur.fetchall()
df = pd.DataFrame(data, columns = ["Region","Total_Sales"])
df

,Region,Total_Sales
0,Technology,827455.87
1,Furniture,728658.57
2,Office Supplies,705422.33
